<a href="https://colab.research.google.com/github/skhazaei/TensorFlow-repo/blob/master/metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's first build a sample model with `metrics=['accuracy']`

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential

In [2]:
model = Sequential([
                    Flatten(input_shape=(28,28)),
                    Dense(32, activation='relu'),
                    Dense(32, activation='tanh'),
                    Dense(10, activation='softmax')
])

In [4]:
model.compile(optimizer='adam',
              loss='Sparse_categorical_crossentropy',
              metrics=['accuracy'])

# `accuracy` as a metric to judge the performance of the model, how does it work?

## **Case 1: Binary classification with sigmoid activation function**

Let's suppose we are training a model for a binary classification problem. 

Given a training example input $x^{i}$, the model will output a float number between 0 and 1. Based on whether this float is less or higher than a threshold (which is by default at 0.5), we round the float number to get a value for $y_{pred}$ from the model. 

**The accuracy** metric compares the value of each $y_{pred}$ on each training example with its true value of one-hot coded vector $y_{true}^{(i)}$ from our training data:
Let 

$$
\delta(y_{pred}^{(i)}, y_{true}^{(i)})=
\begin{cases}
1 & y_{pred}=y_{true} \\
0 & y_{pred}\neq y_{true}
\end{cases}
$$

The **accuracy metric compute the mean value of** $\delta(y_{pred}^{(i)}, y_{true}^{(i)})$ **over all training example:**

$$accuracy = \frac{1}{N}\sum_{i=1}^{N} \delta(y_{pred}^{(i)}, y_{true}^{(i)})$$

The following is the implementation for an example in the backend of keras:

In [37]:
import tensorflow as tf 
import tensorflow.keras.backend as K
# Sigmoid activation function
## tf.keras.backend.equal: Element-wise equality between two tensors.

y_true = tf.constant([0.0,1.0,1.0])
y_pred = tf.constant([0.4,0.8, 0.3])
accuracy = K.mean(K.equal(y_true, K.round(y_pred)))
accuracy

<tf.Tensor: shape=(), dtype=float32, numpy=0.6666667>

## **Case 2: Categorical classification with softmax activation function**

Now let's suppose that we are training a classification model that sorts out the data into m classes, $m>2$, using softmax function in the last layer.

Given a training example $x^{(i)}$, the model outputs a tensor of probabilities $p_1, p_2, ..., p_m$, giving the likelihood that the  training examples belong into each class.

The accuracy metric works by determining the largest argument in the $y_{pred}^{(i)}$ tensor, and compares its index to the index of the maximum value of $y_{true}^{(i)}$ to determine $\delta(y_{pred}^{(i)}, y_{true}^{(i)})$. It then computes the accuracy in the same way as for the binary classification case.

$$ accuracy = \frac{1}{N} \sum_{i=1}^N \delta(y_{pred}^{(i)},y_{true}^{(i)}) $$

The following is the implementation for an example in the backend of keras:

In [35]:
# Binary classification with softmax

y_true = tf.constant([[0.0,1.0],[1.0,0.0],[1.0,0.0],[0.0,1.0]])
y_pred = tf.constant([[0.4,0.6], [0.3,0.7], [0.05,0.95],[0.33,0.67]])
accuracy =K.mean(K.equal(y_true, K.round(y_pred)))
accuracy

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [34]:
# Categorical classification with m>2

y_true = tf.constant([[0.0,1.0,0.0,0.0],[1.0,0.0,0.0,0.0],[0.0,0.0,1.0,0.0]])
y_pred = tf.constant([[0.4,0.6,0.0,0.0], [0.3,0.2,0.1,0.4], [0.05,0.35,0.5,0.1]])
accuracy = K.mean(K.equal(K.argmax(y_true, axis=-1), K.argmax(y_pred, axis=-1)))
accuracy

<tf.Tensor: shape=(), dtype=float32, numpy=0.6666667>

# **`accuracy` vs. `binary_accuracy` vs. `categorical_accuracy`**

the `binary_accuracy` and `categorical_accuracy` metrics are, by default identical to the case 1 and 2 respectively of the `accuracy` metric explained above.

However, using `binary_accuracy` allows you to use the optional `threshold` argument, which sets the minimum value of $y_{pred}$ which will be rounded to 1. `threshold=0.5` is by default.

```
tf.keras.metrics.BinaryAccuracy(
    name='binary_accuracy', dtype=None, threshold=0.5
)
```
```
tf.keras.metrics.CategoricalAccuracy(
    name='categorical_accuracy', dtype=None
)
```

Below we give some examples of how to compile a model with `binary_accuracy` with and without a threshold.

In [38]:
# Compile the model with default threshold (=0.5)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['binary_accuracy'])

In [39]:
# The threshold can be specified as follows

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5)])

# Sparse categorical accuracy 

This is very similar metric to `categorical_accuracy` with one major difference - The label $y_{true}$ of each training example is not expected to be one-hot encoded vector, but to be a tensor consisting of a single integer. This integer is then compared to the index of the maximum argument of $y_{pred}$ to determine $\delta(y_{pred}^{(i)},y_{true}^{(i)})$.

```
tf.keras.metrics.SparseCategoricalAccuracy(
    name='sparse_categorical_accuracy', dtype=None
)
```

In [40]:
# Two examples of compiling a model with a sparse categorical accuracy metric

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=["sparse_categorical_accuracy"])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# **(Sparse) Top $k$-categorical accuracy** :

In top $k$-categorical accuracy, instead of computing how often the model correctly predicts the label of a training example, the metric computes how often the model has $y_{true}$ in the top $k$ of its predictions. By default, $k=5$.

As before, the main difference between top $k$-categorical accuracy and its sparse version is that the former assumes $y_{true}$ is a one-hot encoded vector, whereas the sparse version assumes $y_{true}$ is an integer.

```
tf.keras.metrics.SparseTopKCategoricalAccuracy(
    k=5, name='sparse_top_k_categorical_accuracy', dtype=None
)
```

```
tf.keras.metrics.TopKCategoricalAccuracy(
    k=5, name='top_k_categorical_accuracy', dtype=None
)
```


In [41]:
# Compile a model with a top-k categorical accuracy metric with default k (=5)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=["top_k_categorical_accuracy"])

In [42]:
# Specify k instead with the sparse top-k categorical accuracy

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3)])

## Custom metrics
It is also possible to define your own custom metric in Keras.
You will need to make sure that your metric takes in (at least) two arguments called `y_true` and `y_pred` and then output a single tensor value.

In [43]:
# Define a custom metric

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

In [45]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=[mean_pred])

## Multiple metrics
Finally, it is possible to use multiple metrics to judge the performance of your model. 


Here's an example:

In [47]:
# Compile the model with multiple metrics

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=[mean_pred,
                       'accuracy',
                       tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3)])